In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
from pathlib import Path
from mqt.bench.evaluation import evaluation

font = {"size": 12}
matplotlib.rc("font", **font)

In [ ]:
evaluation.create_statistics()

In [ ]:
with Path("./evaluation_data.pkl").open("rb") as f:
    eval_data = pickle.load(f)
df = pd.DataFrame(eval_data)
df

In [ ]:
df = df.join(pd.json_normalize(df["supermarq_features"])).drop("supermarq_features", axis="columns")
df

In [ ]:
benchmarks = df["filename"].values
num_qubits = df["num_qubits"].values
depth = df["depth"].values
num_gates = df["num_gates"].values
num_multiple_qubit_gates = df["num_multiple_qubit_gates"].values
program_communication = df["program_communication"].values
critical_depth = df["critical_depth"].values
entanglement_ratio = df["entanglement_ratio"].values
parallelism = df["parallelism"].values
liveness = df["liveness"].values
perc_multi = num_multiple_qubit_gates / num_gates

# Device Distribution

In [ ]:
overall_count_mapped = evaluation.count_occurrences(benchmarks, "mapped")
mapped_ibm_montreal_count = evaluation.count_occurrences(benchmarks, "mapped_ibm_montreal")
mapped_ibm_washington_count = evaluation.count_occurrences(benchmarks, "mapped_ibm_washington")
mapped_oqc_lucy_count = evaluation.count_occurrences(benchmarks, "mapped_oqc_lucy")
mapped_rigetti_count = evaluation.count_occurrences(benchmarks, "mapped_rigetti")
mapped_ionq_count = evaluation.count_occurrences(benchmarks, "mapped_ionq")

qiskit_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "qiskit")
tket_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "tket")

In [ ]:
perc_montreal = mapped_ibm_montreal_count / overall_count_mapped
perc_washington = mapped_ibm_washington_count / overall_count_mapped
perc_oqc = mapped_oqc_lucy_count / overall_count_mapped
perc_rigetti = mapped_rigetti_count / overall_count_mapped
perc_ionq = mapped_ionq_count / overall_count_mapped

In [ ]:
data = [perc_washington, perc_rigetti, perc_montreal, perc_ionq, perc_oqc]
colors = ["#B7E6A5", "#7CCBA2", "#46AEA0", "#089099", "#00718B"]
labels = ["IBMQ Washington (127)", "Rigetti Aspen M2 (80)", "IBMQ Montreal (27)", "IonQ Harmony (11)", "OQC Lucy (8)"]
pies = plt.pie(data, labels=labels, autopct="%1.1f%%", colors=colors)
for pie in pies[0]:
    pie.set_alpha(0.6)
plt.savefig("results/pie.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Num Qubits

In [ ]:
qiskit_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "qiskit")
tket_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "tket")

# colors = ['#7CCBA2', '#46AEA0', '#089099', "#00718B", "#045275", "#003147"]
# colors = ['#089099', '#003147']
plt.figure(figsize=(10, 5))
plot = sns.histplot(
    data=[qiskit_qubit_numbers, tket_qubit_numbers],
    bins=128,
    stat="density",
    alpha=0.3,
    kde=True,
    edgecolor="white",
    linewidth=0.5,
    palette=sns.color_palette("viridis", 2),
)
plt.legend(["TKET", "Qiskit"])
plt.ylabel("Relative Frequency")
plt.xlabel("Number of Qubits")
plt.savefig("results/qubit_dist.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Further Characteristics

In [ ]:
datasets = [
    program_communication.astype(float),
    critical_depth.astype(float),
    entanglement_ratio.astype(float),
    liveness.astype(float),
    parallelism.astype(float),
    perc_multi.astype(float),
]
labels = [
    "Program Communication",
    "Critical Depth",
    "Entanglement Ratio",
    "Liveness",
    "Parallelism",
    "Percentage Multi-Qubit Gates",
]

In [ ]:
fig = plt.figure(figsize=(19, 4))
ax = fig.add_subplot(111)
colors = ["#7CCBA2", "#46AEA0", "#089099", "#00718B", "#045275", "#003147"]
pos = range(len(datasets))
plots = plt.violinplot(datasets, pos)
for pc, color in zip(plots["bodies"], colors):
    pc.set_facecolor(color)

ax.set_xticks(pos)
plt.xticks(rotation=0)
ax.set_xticklabels(labels)
plt.savefig("results/violins.pdf", format="pdf", bbox_inches="tight")